In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx

from scipy import stats

import pybel as pb
import json
import time
import csv

import torch
import pyro

from bel2scm import graph_node as cn
from bel2scm import gen_test_data as td
from bel2scm import causal_graph as cg

pyro.set_rng_seed(101)
torch.set_default_tensor_type(torch.DoubleTensor)

In [4]:
tot_data = td.data_gen(10000)

# Initialize Graph from BEL Statements

In [3]:
str_list = ['temp =| cloudy','cloudy => rainy','temp => icream','rainy =| icream']
type_dict = {}
type_dict['temp'] = 'continuous'
type_dict['cloudy'] = 'continuous'
type_dict['rainy'] = 'continuous'
type_dict['icream'] = 'continuous'


In [4]:
graph_test = cg.cg_graph(str_list=str_list,type_dict=type_dict,only_creases=False)

The causal graph is a acyclic


# Initialize Graph from json File

In [5]:
graph_test = cg.cg_graph(json_file='./bel2scm/sample_data/icream.txt',type_dict=type_dict)

The causal graph is a acyclic


# Initialize Graph from PyBEL graph

In [6]:
# bel_temp = pb.from_bel_script('./bel2scm/sample_data/sag_beL_graph.txt')

In [7]:
# graph_test = cg.cg_graph(bel_graph=bel_temp)

# Test Various Graph Capabilities

## Write to file that Can be Read by Causal Fusion

In [8]:
graph_test.write_to_cf('test',100)

/home/somya/miniconda3/lib/python3.7/site-packages/networkx/drawing/layout.py:923: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  pos = np.row_stack((pos[x] for x in node_list))


## Initialize Graph Probabilities

In [9]:
graph_test.prob_init(tot_data)
print(graph_test.exog_list)

temp
tensor(2.7520)

cloudy
tensor(-0.2666, grad_fn=<NegBackward>)

icream
tensor(16.8239, grad_fn=<NegBackward>)

rainy
tensor(-1.8582, grad_fn=<NegBackward>)

['temp']


## Inspect Graph Properties

In [10]:
for item in graph_test.node_dict:
    print(item)
    print(graph_test.node_dict[item].name)
    print(graph_test.node_dict[item].node_type)
    print()

temp
temp
continuous

cloudy
cloudy
continuous

icream
icream
continuous

rainy
rainy
continuous



In [11]:
for item in graph_test.node_dict:
    print(graph_test.node_dict[item].alpha_j)
    print(graph_test.node_dict[item].alpha_jk)
    print(graph_test.node_dict[item].beta_j)
    print(graph_test.node_dict[item].beta_jk)
    print()

tensor(26.1867)
[]
tensor(1.3320)
[]

tensor([0.2666, 0.6749], requires_grad=True)
tensor([[-0.0059]], requires_grad=True)
tensor([0.6993, 1.2256], requires_grad=True)
tensor([[-0.0153]], requires_grad=True)

tensor([0.0761, 1.4155, 0.0801], requires_grad=True)
tensor([[-1.9553e-02, -3.9454e+00],
        [ 2.0017e+01,  1.9549e+01]], requires_grad=True)
tensor([0.0423, 5.2386, 0.0632], requires_grad=True)
tensor([[-1.5097e-02,  6.6157e-01],
        [ 2.0433e+01,  2.1594e+01]], requires_grad=True)

tensor([-0.2090,  0.2986], requires_grad=True)
tensor([[0.0809]], requires_grad=True)
tensor([-0.9166,  1.2521], requires_grad=True)
tensor([[1.0041]], requires_grad=True)



## Sample the Graph under Various Situations

In [12]:
graph_test.model_sample()

{'temp': tensor(16.3780),
 'cloudy': tensor(0.2291, grad_fn=<SqueezeBackward0>),
 'rainy': tensor(0.0046, grad_fn=<SqueezeBackward0>),
 'icream': tensor(3.3357, grad_fn=<SqueezeBackward0>)}

In [13]:
cond_dict = {}
cond_dict['cloudy'] = torch.Tensor([0.5])

cond_test = graph_test.model_cond_sample(cond_dict)

for item in cond_test:
    print(item)
    print(cond_test[item])
    print()

temp
tensor(22.8293)

cloudy
tensor(0.5000)

rainy
tensor(0.0001, grad_fn=<SqueezeBackward0>)

icream
tensor(1.1930, grad_fn=<SqueezeBackward0>)



In [14]:
do_dict = {}
do_dict['rainy'] = torch.Tensor([2.5])

do_test = graph_test.model_do_sample(do_dict)

for item in do_test:
    print(item)
    print(do_test[item])
    print()



temp
tensor(14.0725)

cloudy
tensor(0.1235, grad_fn=<SqueezeBackward0>)

rainy
tensor(2.5000)

icream
tensor(0.7328, grad_fn=<SqueezeBackward0>)



In [15]:
do_cond_test = graph_test.model_do_cond_sample(do_dict,cond_dict)
for item in do_cond_test:
    print(item)
    print(do_cond_test[item])
    print()

temp
tensor(18.0229)

cloudy
tensor(0.5000)

rainy
tensor(2.5000)

icream
tensor(0.7942, grad_fn=<SqueezeBackward0>)



In [16]:
obs_dict = {}
obs_dict['icream'] = torch.Tensor([0.5])
obs_dict['rainy'] = torch.Tensor([0.8])

do_dict = {}
do_dict['rainy'] = torch.Tensor([1.5])


counter_test = graph_test.model_counterfact(obs_dict,do_dict)
for item in counter_test:
    print(item)
    print(counter_test[item])
    print()


temp
tensor(24.2905)

cloudy
tensor(0.5170, grad_fn=<SqueezeBackward0>)

rainy
tensor(1.5000)

icream
tensor(0.7630, grad_fn=<SqueezeBackward0>)



## Do Statistical Tests on Data and Model

In [17]:
print(graph_test.cond_mut_info(['icream'],['rainy'],['temp'],tot_data))

0.15610234126017097


In [18]:
a = graph_test.g_test(['icream'],tot_data)
print(a)

(tensor(nan), nan)


## Calculate Various Effect Levels

In [20]:
do_dict1 = {}
do_dict1['cloudy'] = torch.tensor([0.9])
do_dict2 = {}
do_dict2['cloudy'] = torch.tensor([0.1])

target = ['icream']

print(graph_test.tot_effect(target,do_dict1,do_dict2,10000))

{'icream': -1.0689832039220977}


In [21]:
do_dict3 = {}
do_dict3['rainy'] = torch.tensor([2.5])
do_dict4 = {}
do_dict4['rainy'] = torch.tensor([0.5])

med_dict = {}
med_dict['temp'] = torch.tensor([25.0])

print(graph_test.cd_effect(target,do_dict3,do_dict4,med_dict,10000))

{'icream': 0.023117164014903935}


In [22]:
print(graph_test.nd_effect(target,do_dict1,do_dict2,10000))

{'icream': -0.49505771394724296}


In [23]:
print(graph_test.ni_effect(target,do_dict1,do_dict2,10000))

{'icream': 2.6659770394411484}


### testing on covid graph

In [2]:
# bel_temp = pb.from_bel_script('./COVID-19-new.json')
graph_test = cg.cg_graph(nano_pub= './COVID-19-new.json')

The causal graph is a acyclic
BEL node type deg not known -- defaulting to continuous


In [5]:
## initialize graph probabilities
graph_test.prob_init(tot_data)
print(graph_test.exog_list)

IndexError: index 5 is out of bounds for dimension 0 with size 4